<a href="https://www.kaggle.com/code/vaishnavrathod50/chess-winner-prediction-by-rnn?scriptVersionId=166828671" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# importing libraries for Dataset
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv('/kaggle/input/chess/games.csv')
data.head()

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [3]:
data.shape

(20058, 16)

In [4]:
data.columns

Index(['id', 'rated', 'created_at', 'last_move_at', 'turns', 'victory_status',
       'winner', 'increment_code', 'white_id', 'white_rating', 'black_id',
       'black_rating', 'moves', 'opening_eco', 'opening_name', 'opening_ply'],
      dtype='object')

In [5]:
data['winner'].unique()

array(['white', 'black', 'draw'], dtype=object)

In [6]:
moves = np.array(data.query("winner != 'draw'")['moves'])
labels = np.array(data.query("winner != 'draw'")['winner'].apply(lambda x: 1 if x == 'white' else 0))

print(moves)
print(labels)

['d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5 Bf4'
 'd4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6 Qe5+ Nxe5 c4 Bb4+'
 'e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc6 bxc6 Ra6 Nc4 a4 c3 a3 Nxa3 Rxa3 Rxa3 c4 dxc4 d5 cxd5 Qxd5 exd5 Be6 Ra8+ Ke7 Bc5+ Kf6 Bxf8 Kg6 Bxg7 Kxg7 dxe6 Kh6 exf7 Nf6 Rxh8 Nh5 Bxh5 Kg5 Rxh7 Kf5 Qf3+ Ke6 Bg4+ Kd6 Rh6+ Kc5 Qe3+ Kb5 c4+ Kb4 Qc3+ Ka4 Bd1#'
 ...
 'd4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd2 b6 Ne5 Nxe5 Bxe5 Nd7 Bxh7+ Kxh7 Qh5+ Kg8 Nf3 f6 Bf4 g5 Qg6+ Kh8 Nh4 Qe8 Qh6+ Kg8 Ng6 Kf7 Qh7#'
 'e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb6 c5 Nd5 Bc4 e6 Bxd5 exd5 Nc3 d4 Ne4 Bf5 f3 Nd7 b4 Nxe5 Bf4 f6 g4 Bxe4 fxe4 c6 Bxe5 fxe5 Nf3 Be7 Nxe5 Bf6 Nc4 O-O-O h4 h6 e5 b5 cxb6 Be7 bxa7 Kc7 a3 Rhf8 Kd2 Rf4 Rag1 d3 h5 Rf2+ Ke3 Re2+ Kf4 Rf8+ Kg3 Re3+ Nxe3 d2 Rd1 Bg5 Nf5 Kb7 Rhf1 Kxa7 Nd6 Rxf1 Rxf1 Kb6 e6 Kc7 Nf5 Kc8 e7 Kd7 a4 Bxe7 Nxe7 Kxe7 a5 Kd7 Rd1 Kc7 Rxd2 Kb7 Ra2 Ka6 Kf4 Kb5 a6 Kb6 a7 Kb5 a8=Q Kxb4 Qxc6 g5+ hxg6 Kb3 Rc2 Kb4 Qb7+ Ka3 Rc8'
 'd4 d5 Bf4 Na6 e3 e6

In [7]:
# getting all unique moves

unique_moves = []

for move_list in moves:
    for move in move_list.split(" "):
        if move not in unique_moves:
            unique_moves.append(move)

len(unique_moves)

4373

In [8]:
# Getting the longest moves for a game played

len_of_moves = []
for move_list in moves:
    len_of_moves.append(len(move_list.split(" ")))

max(len_of_moves)

349

In [9]:
# Creating input vectors
# Installing libraries for above

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split


2024-03-13 12:53:44.616962: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 12:53:44.617064: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 12:53:44.761688: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
tok = Tokenizer(num_words=len(unique_moves))
tok.fit_on_texts(moves)

sequences = tok.texts_to_sequences(moves)
word_index = tok.word_index

model_inputs = pad_sequences(sequences=sequences, maxlen=max(len_of_moves))  # All the elements in array becomes of smae len
model_inputs

array([[  0,   0,   0, ...,   9, 363,  58],
       [  0,   0,   0, ...,  41,  11,  60],
       [  0,   0,   0, ..., 202, 521, 563],
       ...,
       [  0,   0,   0, ..., 194,  86, 528],
       [  0,   0,   0, ..., 360, 505,  85],
       [  0,   0,   0, ..., 247, 177, 280]], dtype=int32)

In [11]:
model_inputs.shape, labels.shape

((19108, 349), (19108,))

In [12]:
# We can start training our model

X_train, X_test, y_train, y_test = train_test_split(model_inputs, labels, test_size=0.33, random_state=42)

In [13]:
max(len_of_moves)

349

In [14]:
# Building RNN Model

inputs = tf.keras.layers.Input(shape=(max(len_of_moves),))

embedding = tf.keras.layers.Embedding(
    input_dim = len(unique_moves),
    output_dim = 256, # Embedding dimension
    #input_len = max(len_of_moves)
)(inputs)

gru = tf.keras.layers.GRU(units = 256)(embedding)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(gru)

model = tf.keras.Model(inputs = inputs, outputs = outputs)

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy', 
                tf.keras.metrics.AUC(name='auc')
               ]
)

history = model.fit(X_train, y_train, 
                  validation_split = 0.2, 
                  batch_size = 32, epochs = 5,
                  callbacks = tf.keras.callbacks.ReduceLROnPlateau()
                  )


Epoch 1/5
321/321 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.6793 - auc: 0.7465 - loss: 0.5890 - val_accuracy: 0.7864 - val_auc: 0.8648 - val_loss: 0.4589 - learning_rate: 0.0010
Epoch 2/5
321/321 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.8234 - auc: 0.9020 - loss: 0.3959 - val_accuracy: 0.8665 - val_auc: 0.9388 - val_loss: 0.3138 - learning_rate: 0.0010
Epoch 3/5
321/321 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.8906 - auc: 0.9564 - loss: 0.2674 - val_accuracy: 0.8614 - val_auc: 0.9400 - val_loss: 0.3173 - learning_rate: 0.0010
Epoch 4/5
321/321 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.9083 - auc: 0.9696 - loss: 0.2230 - val_accuracy: 0.8462 - val_auc: 0.9288 - val_loss: 0.3959 - learning_rate: 0.0010
Epoch 5/5
321/321 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.9175 - auc: 0.9773 - loss: 0.1940 - val_accuracy: 0.8946 - val_auc: 0.9590 - val_loss: 0.2690 - learning_rate: 0.0010


In [15]:
import plotly as px

In [16]:
fig = px.plot(
    history.history, 
    y = ['loss', 'val_loss'],
    labels = {'x':'Epochs', 'y':'Loss'},
    title= 'Loss over time.',
    kind='line'
             )
fig.show()



In [17]:
fig = px.plot(
    history.history, 
    y = ['auc', 'val_auc'],
    labels = {'x':'Epochs', 'y':'AUC'},
    title= 'AUC over time.',
    kind = 'line'
             )
fig.show()

In [18]:
model.evaluate(X_test, y_test)

198/198 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8953 - auc: 0.9573 - loss: 0.2739


[0.2856367230415344, 0.8908975720405579, 0.9540355801582336]